# Task 1

# 1. Maintain a running conversation history of user–assistant chats.

In [ ]:
import time
from dataclasses import dataclass, field
from typing import List, Dict
from openai import OpenAI
from datetime import datetime

In [ ]:
@dataclass
class message:
  role : str
  content : str
  Time : float = field(default_factory=time.time)

  def seralize(self) -> dict:

    return {
        "role" :self.role,
        "content" : self.content,
        "Time" :self.Time
    }

class ConversationHistory:

  def __init__(self):

    self.history : list[message] = []

  def add_message(self,role:str,content:str):
    self.history.append(message(role=role,content=content))

  def get_open_ai_format(self):
   return [{'role':m.role ,"content":m.content} for m in self.history]

  def print_history(self):
    print("\n--- Conversation History --- \n")

    for m in self.history:
        ts = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(m.Time))
        print(f"{ts} | {m.role.upper()}: {m.content} \n")

    print("--- END ---\n")



# Demo 0


In [ ]:

chat = ConversationHistory()

chat.add_message("user", "Hello!")
chat.add_message("assistant", "Hi there!")

chat.print_history()


--- Conversation History --- 

2025-09-16 11:36:35 | USER: Hello! 

2025-09-16 11:36:35 | ASSISTANT: Hi there! 

--- END ---



# Demo 1


In [ ]:
client = OpenAI(
    api_key="",
    base_url="https://api.groq.com/openai/v1" )


In [ ]:
# system context
chat.add_message("system", "You are a sarcastic Gen Z data science tutor.")

# user question
chat.add_message("user", "Explain YOY growth in simple words.")

In [ ]:
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=chat.get_open_ai_format()
)

In [ ]:
assistant_reply = response.choices[0].message.content
chat.add_message("assistant", assistant_reply)

chat.print_history()


--- Conversation History --- 

2025-09-16 11:36:35 | USER: Hello! 

2025-09-16 11:36:35 | ASSISTANT: Hi there! 

2025-09-16 11:36:35 | SYSTEM: You are a sarcastic Gen Z data science tutor. 

2025-09-16 11:36:35 | USER: Explain YOY growth in simple words. 

2025-09-16 11:36:37 | ASSISTANT: So, you want to learn about YOY growth? Like, it's not that hard. YOY stands for Year-Over-Year. It's a way to measure how much something (like sales, revenue, or users) has changed from one year to the next.

Think of it like this: let's say your favorite TikTok creator had 1 million followers last year and now they have 1.2 million. That's a 20% YOY growth, because their followers increased by 20% from last year to this year.

Got it? It's not rocket science. 

--- END ---



# Demo 2


In [ ]:

chat = ConversationHistory()


chat.add_message("system", "You are a Gen Z data science tutor, keep it short and witty.")


samples = [
    "Hey, can you help me clean my Power BI dashboard?",
    "Also, why is my PBIX not uploading to Drive?",
    "How do I record Power BI screen with Game Bar?",
    "My laptop is low on space, what can I delete safely?",
    "For a card visual, how do I show green up and red down arrows?",
    "Return rate YOY calculation is showing wrong; fix the DAX."
]


for msg in samples:
    chat.add_message("user", msg)


    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=chat.get_open_ai_format()
    )

    assistant_reply = response.choices[0].message.content
    chat.add_message("assistant", assistant_reply)


chat.print_history()



--- Conversation History --- 

2025-09-16 11:36:37 | SYSTEM: You are a Gen Z data science tutor, keep it short and witty. 

2025-09-16 11:36:37 | USER: Hey, can you help me clean my Power BI dashboard? 

2025-09-16 11:36:37 | ASSISTANT: Lowkey, a messy dashboard can be so extra. What's going on with it? Too many visuals, sluggish performance, or just a hot mess? Spill the tea, and we'll get it looking fire in no time! 

2025-09-16 11:36:37 | USER: Also, why is my PBIX not uploading to Drive? 

2025-09-16 11:36:38 | ASSISTANT: Upload struggles are the worst! Maybe your file is too large or has some rogue data models. Try checking the file size, disabling any unnecessary visuals, and simplifying those data models. If that doesn't work, we can dive deeper into the issue. Also, are you using the correct export settings and file format (PBIX, not PBIT, right?)? 

2025-09-16 11:36:38 | USER: How do I record Power BI screen with Game Bar? 

2025-09-16 11:36:38 | ASSISTANT: Sick tutorial goal

#  Implement summarization of conversation history to keep it concise.

In [ ]:
def summarize_history(chat_history: ConversationHistory, model_client=None):

    text_blob = ""
    for msg in chat_history.history:
        text_blob += f"[{msg.role.upper()}] {msg.content}\n"


    prompt = (
        "Summarize the following conversation concisely, keeping key points and roles:\n\n"
        + text_blob
    )

    if model_client is None:

        summary = "\n".join(text_blob.split("\n")[:3])

    else:
        response = model_client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}]
        )
        summary = response.choices[0].message.content

    return summary



In [ ]:
chat = ConversationHistory()
chat.add_message("system", "You are a Gen Z witty data tutor.")

# Feed multiple user messages
samples = [
    "Hey, can you help me clean my Power BI dashboard?",
    "Also, why is my PBIX not uploading to Drive?",
    "How do I record Power BI screen with Game Bar?",
    "My laptop is low on space, what can I delete safely?",
]

for msg in samples:
    chat.add_message("user", msg)
    chat.add_message("assistant", f"Ai: {msg[:50]}")


# Demo

In [ ]:
summary = summarize_history(chat,client )
print("=== Summarized Conversation ===")
print(summary)

=== Summarized Conversation ===
A Gen Z data tutor had a conversation with a user. The user asked for help with: 
1. Cleaning their Power BI dashboard
2. Troubleshooting a PBIX upload issue to Drive
3. Recording a Power BI screen using Game Bar
4. Freeing up laptop space by safely deleting files.


# Customization options for truncation

  


In [ ]:
import copy
@dataclass
class Message:
    role: str
    content: str

    def serialize(self):
        """Only role + content are sent to Groq."""
        return {"role": self.role, "content": self.content}


class ConversationHistory:
    def __init__(self, default_model="llama-3.1-8b-instant"):
        self.history: List[Message] = []
        self.default_model = default_model


    def add_message(self, role: str, content: str):
        self.history.append(Message(role=role, content=content))


    def get_truncated_by_turns(self, last_n: int) -> List[Message]:
        if last_n <= 0:
            return []
        return copy.deepcopy(self.history[-last_n:])

    def get_truncated_by_chars(self, max_chars: int) -> List[Message]:
        total = 0
        new_hist = []
        for msg in reversed(self.history):
            length = len(msg.content)
            if total + length > max_chars and new_hist:
                break
            new_hist.append(msg)
            total += length
        return list(reversed(new_hist))

    def get_truncated_by_words(self, max_words: int) -> List[Message]:
        total = 0
        new_hist = []
        for msg in reversed(self.history):
            wc = len(msg.content.split())
            if total + wc > max_words and new_hist:
                break
            new_hist.append(msg)
            total += wc
        return list(reversed(new_hist))


    def get_serialized_history(self, use_truncated: List[Message] = None):
        if use_truncated is not None:
            return [m.serialize() for m in use_truncated]
        return [m.serialize() for m in self.history]


    def ask_model(self, use_truncated: List[Message] = None, model: str = None):

        chosen_model = model or self.default_model  # use small model by default
        response = client.chat.completions.create(
            model=chosen_model,
            messages=self.get_serialized_history(use_truncated)
        )
        reply = response.choices[0].message.content
        self.add_message("assistant", reply)
        return reply

    def print_history(self, history_to_print: List[Message] = None, show_timestamp=False):
        hist = history_to_print if history_to_print is not None else self.history
        print("\n--- Conversation History ---")
        for m in hist:

            ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S") if show_timestamp else ""
            ts_part = f"{ts} | " if show_timestamp else ""
            print(f"{ts_part}{m.role.upper()}: {m.content}")
        print("--- END ---\n")




In [ ]:
conv = ConversationHistory()

samples = [
    "Hey, how are you?",
    "Can you summarize a short story for me?",
    "Once upon a time, there was a cat who wanted to fly.",
    "Interesting! Can you tell me more about the cat's adventures?",
    "Sure, the cat tried to build wings from leaves and sticks."
]

In [ ]:
for msg in samples:
    conv.add_message("user", msg)

    conv.add_message("assistant", f"Echo: {msg}")


# Demo 1

In [ ]:
print("=== LAST 3 TURNS ===")
trunc_turns = conv.get_truncated_by_turns(3)
conv.print_history(trunc_turns)

=== LAST 3 TURNS ===

--- Conversation History ---
ASSISTANT: Echo: Interesting! Can you tell me more about the cat's adventures?
USER: Sure, the cat tried to build wings from leaves and sticks.
ASSISTANT: Echo: Sure, the cat tried to build wings from leaves and sticks.
--- END ---



# Demo 2

In [ ]:
trunc_turns = conv.get_truncated_by_turns(5)
conv.print_history(trunc_turns)


--- Conversation History ---
ASSISTANT: Echo: Once upon a time, there was a cat who wanted to fly.
USER: Interesting! Can you tell me more about the cat's adventures?
ASSISTANT: Echo: Interesting! Can you tell me more about the cat's adventures?
USER: Sure, the cat tried to build wings from leaves and sticks.
ASSISTANT: Echo: Sure, the cat tried to build wings from leaves and sticks.
--- END ---



# Demo 1

In [ ]:
print("=== LAST 100 CHARACTERS ===")
trunc_chars = conv.get_truncated_by_chars(100)
conv.print_history(trunc_chars)

=== LAST 100 CHARACTERS ===

--- Conversation History ---
ASSISTANT: Echo: Sure, the cat tried to build wings from leaves and sticks.
--- END ---



# Demo 2

In [ ]:
print("=== LAST 500 CHARACTERS ===")
trunc_chars = conv.get_truncated_by_chars(500)
conv.print_history(trunc_chars)

=== LAST 500 CHARACTERS ===

--- Conversation History ---
USER: Hey, how are you?
ASSISTANT: Echo: Hey, how are you?
USER: Can you summarize a short story for me?
ASSISTANT: Echo: Can you summarize a short story for me?
USER: Once upon a time, there was a cat who wanted to fly.
ASSISTANT: Echo: Once upon a time, there was a cat who wanted to fly.
USER: Interesting! Can you tell me more about the cat's adventures?
ASSISTANT: Echo: Interesting! Can you tell me more about the cat's adventures?
USER: Sure, the cat tried to build wings from leaves and sticks.
ASSISTANT: Echo: Sure, the cat tried to build wings from leaves and sticks.
--- END ---



# Demo 1


In [ ]:
print("=== LAST 20 WORDS ===")
trunc_words = conv.get_truncated_by_words(20)
conv.print_history(trunc_words)

=== LAST 20 WORDS ===

--- Conversation History ---
ASSISTANT: Echo: Sure, the cat tried to build wings from leaves and sticks.
--- END ---



# Demo 2

In [ ]:
print("=== LAST 20 WORDS ===")
trunc_words = conv.get_truncated_by_words(50)
conv.print_history(trunc_words)

=== LAST 20 WORDS ===

--- Conversation History ---
USER: Interesting! Can you tell me more about the cat's adventures?
ASSISTANT: Echo: Interesting! Can you tell me more about the cat's adventures?
USER: Sure, the cat tried to build wings from leaves and sticks.
ASSISTANT: Echo: Sure, the cat tried to build wings from leaves and sticks.
--- END ---



# 4. Add periodic summarization:

In [ ]:

@dataclass
class Message:
    role: str
    content: str

    def serialize(self):
        return {"role": self.role, "content": self.content}


# ----- Conversation summaries and optional display -----
class ConversationHistory:
    def __init__(self, default_model="llama-3.1-8b-instant", summarize_every_k=2, summary_window=2):
        self.history: List[Message] = []
        self.default_model = default_model
        self.summarize_every_k = summarize_every_k
        self.turn_counter = 0
        self.summary_window = summary_window  # last N messages for summary
        self.summary_history: List[str] = []  # track TL;DR summaries

    # Add user message
    def add_user(self, content: str):
        self.history.append(Message(role="user", content=content))

    # Ask model and store reply
    def ask_model(self, user_input: str):
        self.add_user(user_input)

        # Send conversation to API
        response = client.chat.completions.create(
            model=self.default_model,
            messages=[m.serialize() for m in self.history]
        )

        reply = response.choices[0].message.content
        self.history.append(Message(role="assistant", content=reply))

        # Increment turn and summarize if needed
        self.turn_counter += 1
        if self.turn_counter % self.summarize_every_k == 0:
            self._summarize_history()

        return reply

    # 1-line TL;DR summary (safe for API)
    def _summarize_history(self):
        messages_to_summarize = self.history[-self.summary_window:]

        summary_prompt = (
            "Summarize the following conversation in ONE super short sentence "
            "(max 10 words), no extra details, only main point:\n\n"
        )
        for msg in messages_to_summarize:
            summary_prompt += f"{msg.role}: {msg.content}\n"

        response = client.chat.completions.create(
            model=self.default_model,
            messages=[{"role": "user", "content": summary_prompt}]
        )

        summary_text = response.choices[0].message.content.strip()
        summary_text = summary_text.replace("\n", " ")
        if '.' in summary_text:
            summary_text = summary_text.split('.')[0]
        summary_text = summary_text[:100]

        # Store separately, not in API history
        self.summary_history.append(summary_text)

        print("\n--- TL;DR Summary ---")
        print(summary_text)
        print("--- END SUMMARY ---\n")

    # Print conversation or summaries only
    def print_history(self, summaries_only=False):
        if summaries_only:
            print("\n--- TL;DR Summaries Only ---")
            for i, s in enumerate(self.summary_history, 1):
                print(f"Summary {i}: {s}")
            print("--- END ---\n")
            return

        print("\n--- Full Conversation ---")
        for msg in self.history:
            print(f"{msg.role.upper()}: {msg.content}")
        if self.summary_history:
            print("\n--- Summaries ---")
            for i, s in enumerate(self.summary_history, 1):
                print(f"Summary {i}: {s}")
        print("--- END ---\n")


# Demo

In [ ]:
conv = ConversationHistory(summarize_every_k=2)

conv.ask_model("Hi, how are you?")
conv.ask_model("Can you tell me a short story about a cat?")
conv.ask_model("What did the cat do next?")
conv.ask_model("Make it funny!")
conv.ask_model("Add a twist at the end!")

# Only see summaries
conv.print_history(summaries_only=True)



--- TL;DR Summary ---
A cat named Luna saves her elderly human
--- END SUMMARY ---


--- TL;DR Summary ---
Luna becomes the village's unorthodox gardening expert
--- END SUMMARY ---


--- TL;DR Summaries Only ---
Summary 1: A cat named Luna saves her elderly human
Summary 2: Luna becomes the village's unorthodox gardening expert
--- END ---



In [ ]:
conv.history

[Message(role='user', content='Hi, how are you?'),
 Message(role='assistant', content="I'm functioning properly, thanks for asking. What can I help you with today?"),
 Message(role='user', content='Can you tell me a short story about a cat?'),
 Message(role='assistant', content="Once upon a time, in a small village nestled between rolling hills, there lived a beautiful calico cat named Luna. Luna was a stray who had wandered into the village one day, seeking refuge and warmth. She quickly won over the hearts of the villagers, particularly an elderly woman named Elara, who welcomed Luna into her cozy cottage.\n\nElara took great care of Luna, feeding her the finest fish and brushing her luscious fur every day. As the days went by, Luna grew fonder of Elara, who spoke softly to her and shared her stories of adventure. The villagers marveled at the bond between Elara and Luna, who had formed an unbreakable connection.\n\nHowever, one winter night, a fierce storm rolled in, bringing heavy 

In [ ]:

@dataclass
class Message:
    role: str
    content: str

    def serialize(self):
        # API only accepts 'user', 'assistant', 'system'
        return {"role": "system" if self.role == "summary" else self.role, "content": self.content}


class ConversationHistory:
    def __init__(self, default_model="llama-3.1-8b-instant", summarize_every_k=2, summary_window=3):
        self.history: List[Message] = []
        self.default_model = default_model
        self.summarize_every_k = summarize_every_k
        self.summary_window = summary_window
        self.turn_counter = 0

    # Add user message
    def add_user(self, content: str):
        self.history.append(Message(role="user", content=content))

    # Ask model
    def ask_model(self, user_input: str):
        self.add_user(user_input)

        # Send current full history to API
        response = client.chat.completions.create(
            model=self.default_model,
            messages=[m.serialize() for m in self.history]
        )

        reply = response.choices[0].message.content

        # Add assistant message
        self.history.append(Message(role="assistant", content=reply))

        # Increment turn counter and summarize if needed
        self.turn_counter += 1
        if self.turn_counter % self.summarize_every_k == 0:
            self._summarize_history()

        return reply

    # Replace entire conversation history with 1-line TL;DR summary
    def _summarize_history(self):
        messages_to_summarize = self.history[-self.summary_window:]

        prompt = "Summarize this conversation in ONE short sentence (max 10 words):\n"
        for m in messages_to_summarize:
            prompt += f"{m.role}: {m.content}\n"

        response = client.chat.completions.create(
            model=self.default_model,
            messages=[{"role": "user", "content": prompt}]
        )

        summary_text = response.choices[0].message.content.strip()
        summary_text = summary_text.replace("\n", " ")
        if '.' in summary_text:
            summary_text = summary_text.split('.')[0]

        # REPLACE full history with just the summary
        self.history = [Message(role="summary", content=summary_text)]

        print(summary_text)




# Demo

In [ ]:
conv = ConversationHistory(summarize_every_k=2)

conv.ask_model("Hi, how are you?")
conv.ask_model("Tell me a story about a cat.")
conv.ask_model("What happens next?")
conv.ask_model("Make it funny!")

conv.history


I was told a story about a friendship between a cat and a butterfly
Cat and butterfly have hilarious late-night gossip session


[Message(role='summary', content='Cat and butterfly have hilarious late-night gossip session')]

[Errno 2] No such file or directory: 'https://github.com/harshitabasrur/Developer-Internship-Assignment'
/content
